In [6]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from utils.Preprocess import Preprocess

# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents


def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, pad_token_id_for_segment=0,
                                     pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)): 
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            #하나의 단어에 대해서 서브워드로 토큰화 subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            #서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))
            #[CLS]와[SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경 우 max_seq_len-2의 길이로 변환.
            #ex)max_seq_len=64라면 길이가62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
            special_tokens_count = 2
            if len(tokens) > max_seq_len - special_tokens_count:
                tokens = tokens[:(max_seq_len - special_tokens_count)]
                labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]
            #[SEP]를 추가하는 코드
            #1.토큰화 결과의 맨 뒷 부분에[SEP]토큰 추가 #2. 레이블에도 맨 뒷 부분에 -100 추가.
            tokens += [sep_token]
            labels_ids += [pad_token_id_for_label]
            #[CLS]를 추가하는 코드
            #1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
            #2. 레이블의 맨 앞 부분에도 -100 추가.
            tokens = [cls_token] + tokens
            labels_ids = [pad_token_id_for_label] + labels_ids
            #정수 인코딩
            input_id = tokenizer.convert_tokens_to_ids(tokens)
            #어텐션 마스크 생성
            attention_mask = [1] * len(input_id)
            #정수 인코딩에 추가할 패딩 길이 연산 padding_count = max_seq_len - len(input_id)
            #정수 인코딩,어텐션 마스크에 패딩 추가
            input_id = input_id + ([pad_token_id] * padding_count) attention_mask = attention_mask + ([0] * padding_count)
            #세그먼트 인코딩.
            token_type_id = [pad_token_id_for_segment] * max_seq_len
            # 레이블 패딩.(단, 이 경우는 패딩 토큰의 ID가 -100)
            label = labels_ids + ([pad_token_id_for_label] * padding_count)
            assert len(input_id) == max_seq_len, "Error with input length {} vs {}". format(len(input_id), max_seq_len)
            assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
            assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
            assert len(label) == max_seq_len, "Error with labels length {} vs {}". format(len(label), max_seq_len)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)
            data_labels.append(label)

            input_ids = np.array(input_ids, dtype=int) 
            attention_masks = np.array(attention_masks, dtype=int) 
            token_type_ids = np.array(token_type_ids, dtype=int) 
            data_labels = np.asarray(data_labels, dtype=np.int32)

            return (input_ids, attention_masks, token_type_ids), data_labels



p = Preprocess(word2index_dic='../../train_tools/dict/chatbot_dict.bin',
               #userdic='../../utils/user_dic.tsv')
               userdic='../../utils/mtn_user_dict.tsv')

# 학습용 말뭉치 데이터를 불러옴
#corpus = read_file('ner_train.txt')
corpus = read_file('mtn_ner_train_fullversion.txt')
print(corpus[0])


# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)


print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))



# 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=.2,
                                                    random_state=1234)



# 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))

model.add(TFBertModel.from_pretrained(model_name, from_pt=True))
model.add(tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.
                                                initializers.TruncatedNormal
                                                (0.02),
                                            name='classifier'))

model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=1)

print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save('ner_model_use_cpsdic_mtndic_mtndat.h5')


# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result


# f1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report

# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test)    # 실제 NER

# F1 평가 결과
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))


샘플 크기 : 
 65512
0번 째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번 째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 168
샘플 단어 시퀀스 평균 길이 : 8.431951398217121
BIO 태그 사전 크기 : 10
단어 사전 크기 : 17869
index_to_ner {1: 'O', 2: 'B_DT', 3: 'B_FOOD', 4: 'B_LC', 5: 'I', 6: 'B_OG', 7: 'B_PS', 8: 'NNP', 9: 'B_TI', 0: 'PAD'}
학습 샘플 시퀀스 형상 :  (52409, 40)
학습 샘플 레이블 형상 :  (52409, 40, 10)
테스트 샘플 시퀀스 형상 :  (13103, 40)
테스트 샘플 레이블 형상 :  (13103, 40, 10)
410/410 [==============================] - 15s 34ms/step - loss: 0.0508 - accuracy: 0.9832
평가 결과 :  0.9831815361976624
410/410 [==============================] - 14s 32ms/step


c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.war

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       297
           _       0.45      0.38      0.41       640
         _DT       0.99      0.99      0.99     13466
       _FOOD       1.00      1.00      1.00     11685
         _LC       0.94      0.88      0.91      1747
         _OG       0.46      0.40      0.43       464
         _PS       0.64      0.04      0.08       396
         _TI       0.00      0.00      0.00        65

   micro avg       0.97      0.95      0.96     28760
   macro avg       0.69      0.59      0.60     28760
weighted avg       0.97      0.95      0.95     28760

F1-score: 96.3%
